In [ ]:
import os
import numpy as np
np.random.seed(69)
import pandas as pd
import random
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
import tensorflow as tf
from tqdm.notebook import tqdm
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, ZeroPadding2D, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# import kerastuner as kt
# from kerastuner import HyperModel
import time

In [ ]:
def get_metrics(y_pred, y_test, labels, to_print=True):
    correct_labels = np.where(y_pred==y_test)[0]
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred,average='macro')
    recall = metrics.recall_score(y_test, y_pred,average='macro')
    f1score = metrics.f1_score(y_test, y_pred,average='macro')
    # rocscore = metrics.roc_auc_score(y_test, y_pred,average='micro',multi_class="ovo")
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)  
    classification_report = metrics.classification_report(y_test, y_pred)

    if to_print:
        print("Identified {} correct labels out of {} labels".format(len(correct_labels), y_test.shape[0]))
        print("Accuracy:",accuracy)
        print("Precision:",precision)
        print("Recall:",recall)
        print("F1 Score:",f1score)
        # print("ROC AUC Score:",rocscore)
        print(f"Labels are: {labels}")
        print("Confusion Matrix:\n", confusion_matrix)
        print("Classification_Report:\n", classification_report)

    return (correct_labels, accuracy, precision, recall, confusion_matrix, classification_report)

def plot_model_change(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'],label="Training Acc")
    plt.plot(history.history['val_accuracy'],label="Val Acc")
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'],label="Training Loss")
    plt.plot(history.history['val_loss'],label="Val Loss")
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

In [ ]:
df = pd.read_csv("../input/sdss-project-v10/SDSS_Query_v1.0_DF.csv",index_col=0)

regressor = load_model("../input/sdss-project-v10/DNNRegressor.h5")
photodf = df.loc[:,['dered_u', 'deVRad_u', 'psffwhm_u', 'extinction_u',
       'dered_g', 'deVRad_g', 'psffwhm_g', 'extinction_g', 'dered_r',
       'deVRad_r', 'psffwhm_r', 'extinction_r', 'dered_i', 'deVRad_i',
       'psffwhm_i', 'extinction_i', 'dered_z', 'deVRad_z', 'psffwhm_z',
       'extinction_z', 'u_g', 'g_r', 'r_i', 'i_z']]

photodf.loc[:,"redshift"] = regressor.predict(photodf.values)

In [ ]:
X = photodf.values
y = df["class"].values

y, label_strings = pd.factorize(y)
y = to_categorical(y)

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25,random_state=69)

In [ ]:
del(X)

In [ ]:
model = Sequential()

model.add(Dense(1024, input_dim=X_train.shape[1]))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU())
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', verbose=0, patience=100, restore_best_weights=True)
cb = [es]
history = model.fit(X_train, y_train,
                    batch_size=2048,
                    epochs = 4000,
                    validation_data = (X_val,y_val),
                    callbacks = cb,
                    verbose = 2)

In [ ]:
model.save("DNNClassifier.h5")

In [ ]:
plot_model_change(history)

In [ ]:
preds_test = model.predict(X_test,batch_size=2048, verbose = 0)
print(get_metrics(preds_test.argmax(axis=1), y_test.argmax(axis=1),label_strings))

# Identified 27727 correct labels out of 30000 labels
# Accuracy: 0.9242333333333334
# Precision: 0.9248037691383781
# Recall: 0.9242425839531127
# F1 Score: 0.9243388610470348
# Labels are: ['STAR' 'QSO' 'GALAXY']
# Confusion Matrix:
#  [[9105  769  158]
#  [ 466 9142  359]
#  [ 115  406 9480]]

# Identified 27736 correct labels out of 30000 labels
# Accuracy: 0.9245333333333333
# Precision: 0.9249523675725057
# Recall: 0.9245392286702939
# F1 Score: 0.9245957441450002
# Labels are: ['STAR' 'QSO' 'GALAXY']
# Confusion Matrix:
#  [[9113  762  157]
#  [ 463 9120  384]
#  [ 124  374 9503]]